# LSTM

## Load imports.

In [1]:
# Make common scripts visible
import sys
sys.path.append('../common/')

from gensim.models import KeyedVectors
import numpy as np

from loader import load_preprocessed_data
from word_embeddings import DocToIntSequenceConverter
from lstm import LstmPredictor
from lookup_tables import int_to_topic_code, topic_code_to_topic_dict, topic_code_to_int
from sklearn.metrics import classification_report

Using TensorFlow backend.


## Load the data

In [2]:
x, y = load_preprocessed_data('data/rcv1_no_stopwords_reduced.csv')
x = np.array(x)
y = np.array(y)

# Split data into 60% train, 20% validation, 20% test
total_examples = len(y)

split_point_1 = int(total_examples * 0.6)
split_point_2 = int(total_examples * 0.8)

train_x = x[:split_point_1]
train_y = y[:split_point_1]

val_x = x[split_point_1:split_point_2]
val_y = y[split_point_1:split_point_2]

test_x = x[split_point_2:]
test_y = y[split_point_2:]

print('Number of training examples: {}'.format(len(train_x)))

Number of training examples: 1568


## Pre-process the data

In [3]:
# Find the length of a tweet in words
article_lengths = np.array([len(article.split()) for article in train_x])

print('Minimum length of article in words: {}'.format(np.min(article_lengths)))
print('Maximum length of article in words: {}'.format(np.max(article_lengths)))
print('Mean length of article in words: {:.4f}'.format(np.mean(article_lengths)))
print('St dev of length of article in words: {:.4f}'.format(np.std(article_lengths)))

# Set the max sequence length to mean plus 3 standard deviations (99.7% confidence)
max_sequence_length = int(np.mean(article_lengths) + np.std(article_lengths)*3)

# Confirm not many tweets exceed this limit
articles_exceeding_limit = [article for article in train_x if len(article.split()) > max_sequence_length]
percentage_articles_exceeding_limit = (len(articles_exceeding_limit)/len(train_x))*100
print('Percentage of articles exceeding max sequence length limit: {:.4f}%'.format(percentage_articles_exceeding_limit))

Minimum length of article in words: 7
Maximum length of article in words: 519
Mean length of article in words: 112.1582
St dev of length of article in words: 72.5444
Percentage of articles exceeding max sequence length limit: 0.3827%


In [4]:
# Convert articles to sequence of integers representing the words
article_to_int_seq_converter = DocToIntSequenceConverter(x, max_sequence_length)
train_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(train_x)
val_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(val_x)
test_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(test_x)

## Create the LSTM and train it.

In [5]:
word_embedding_dim = 300
word2vec_model = KeyedVectors.load_word2vec_format('../tweet-classification/embeddings/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [6]:
lstm = LstmPredictor(article_to_int_seq_converter.get_word_index(),
                     word_embedding_dim,
                     max_sequence_length,
                     word2vec_model,
                     len(int_to_topic_code.values()))
lstm.train(train_x_seq, train_y, val_x_seq, val_y)

Train on 1568 samples, validate on 523 samples
Epoch 1/100
1568/1568 [==============================] - 52s 33ms/step - loss: 1.2119 - acc: 0.6142 - val_loss: 0.7271 - val_acc: 0.8069
Epoch 2/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.5186 - acc: 0.8495 - val_loss: 0.5613 - val_acc: 0.8394
Epoch 3/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.3553 - acc: 0.8954 - val_loss: 0.3479 - val_acc: 0.8757
Epoch 4/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.3414 - acc: 0.9037 - val_loss: 0.3704 - val_acc: 0.8834
Epoch 5/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.3815 - acc: 0.8929 - val_loss: 0.4155 - val_acc: 0.8700
Epoch 6/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.3134 - acc: 0.9075 - val_loss: 0.3289 - val_acc: 0.9063
Epoch 7/100
1568/1568 [==============================] - 48s 31ms/step - loss: 0.2010 - acc: 0.9483 - val_loss: 0.2932 - val_acc: 0

## Make predictions and report classification accuracy.

In [7]:
# Re-initialise the LSTM, will use weights from the previous training run.
lstm = LstmPredictor(article_to_int_seq_converter.get_word_index(),
                     word_embedding_dim,
                     max_sequence_length,
                     word2vec_model,
                     len(int_to_topic_code.values()),
                     use_saved_weights=True)
test_y_predict = lstm.predict(test_x_seq)
print(classification_report(test_y, test_y_predict, digits=6, target_names=topic_code_to_topic_dict.values()))

                        precision    recall  f1-score   support

CRIME, LAW ENFORCEMENT   0.978495  0.952880  0.965517       191
  ECONOMIC PERFORMANCE   0.966102  0.982759  0.974359        58
             ELECTIONS   0.881356  0.945455  0.912281        55
                HEALTH   0.861111  0.861111  0.861111        36
              RELIGION   0.615385  0.727273  0.666667        11
                SPORTS   1.000000  0.988372  0.994152       172

             micro avg   0.956023  0.956023  0.956023       523
             macro avg   0.883741  0.909641  0.895681       523
          weighted avg   0.958260  0.956023  0.956844       523



## Find examples where predictions went wrong

In [8]:
for topic_code, index in topic_code_to_int.items():
    topic_subset = test_y_predict[test_y == index]
    topic_subset_incorrect = topic_subset[topic_subset != index]
    document_subset = test_x[test_y == index]
    document_subset = document_subset[topic_subset != index]
    
    print('------ 5 random erroneous predictions for {} ------'.format(topic_code_to_topic_dict[topic_code]))
    print('')
    random_indices = np.random.choice(np.arange(len(topic_subset_incorrect)), 5)
    for index in random_indices:
        print(document_subset[index])
        print('')
        print('Above classified as {}'.format(topic_code_to_topic_dict[int_to_topic_code[topic_subset_incorrect[index]]]))
        print('')
    print('')

------ 5 random erroneous predictions for CRIME, LAW ENFORCEMENT ------

Sir_Colin_Hope chairman asbestos-haunted components group N_Plc company possible insurance tools asbestos-linked health liabilities free shareholder value breakthrough imminent Recent press speculation N complex policy liabilities Hope firm interim results news conference premature true imminence press reports misleading breath terms speed solution way speculation new Hope reporters N years anything insurance issues Part way insurance risk rogue cases sort ring-fencing uncertain areas impossible sort economically-affordable premium industry position insurance industry asbestos years Hope data available ability future liabilities asbestos outlook insurance options Reviews situation amenable firm shareholders weight potential asbestos liabilities board objective way shareholder value N shares result sheer uncertainty N results pretax profits stg back well-signalled weaknesses company shares 134-1/2p midday Andrew_Hu